# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee

# Install the pydeck-earthengine-layers package from pip
pip install pydeck-earthengine-layers

jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import ee

## Authentication

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
ee.Authenticate()

### Initialize Earth Engine

The above authentication step creates credentials that are stored on your local computer. Those credentials need to be loaded so that Earth Engine and Pydeck will work.

In [2]:
ee.Initialize()

## Elevation data example

Now let's make a simple example using Shuttle Radar Topography Mission (SRTM) elevation data. Here we create an `ee.Image` object referencing that dataset.

In [3]:
image = ee.Image('CGIAR/SRTM90_V4')

Now we're ready to create the Pydeck layer. The `EarthEngineLayer` makes this simple. Just pass the Earth Engine object to the class.

In [4]:
ee_layer = EarthEngineLayer(image, vis_params={"min": 0, "max": 255})

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [5]:
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)
r = pdk.Deck(
    layers=[ee_layer], 
    initial_view_state=view_state
)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…

## Hillshade Example

As a slightly more in-depth example, let's use the SRTM dataset to calculate hillshading. This example comes from 

In [6]:
# From https://github.com/giswqs/earthengine-py-notebooks/blob/master/Visualization/hillshade.ipynb
# Add Earth Engine dataset
import math

def Radians(img):
  return img.toFloat().multiply(math.pi).divide(180)

def Hillshade(az, ze, slope, aspect):
  """Compute hillshade for the given illumination az, el."""
  azimuth = Radians(ee.Image(az))
  zenith = Radians(ee.Image(ze))
  # Hillshade = cos(Azimuth - Aspect) * sin(Slope) * sin(Zenith) +
  #     cos(Zenith) * cos(Slope)
  return (azimuth.subtract(aspect).cos()
          .multiply(slope.sin())
          .multiply(zenith.sin())
          .add(
              zenith.cos().multiply(slope.cos())))

terrain = ee.Algorithms.Terrain(ee.Image('srtm90_v4'))
slope_img = Radians(terrain.select('slope'))
aspect_img = Radians(terrain.select('aspect'))

In [7]:
ee_object = Hillshade(0, 60, slope_img, aspect_img)
ee_layer = EarthEngineLayer(ee_object)
view_state = pdk.ViewState(latitude=36.0756, longitude=-111.9987, zoom=7, bearing=0, pitch=30)
r = pdk.Deck(
    layers=[ee_layer], 
    initial_view_state=view_state
)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…